## 대성에너지 주가 동향 Daily Report
+ 작성: 임경호

### 1. 주식 정보 가져오기 (출처: fnGuide)

In [1]:
import time
start = time.time()

import requests 
from bs4 import BeautifulSoup
from datetime import datetime
from dateutil.relativedelta import relativedelta

ticker = '117580'   # 대성에너지

# fnGuide 주식 정보
url = f"http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{ticker}"
resp = requests.get(url)

if resp.status_code == 200:
    soup = BeautifulSoup(resp.text, "lxml")
    
    # 시세현황 기준일자
    selector_ref_date = '#div1 > div.um_topbar.botbd_n.bmshadow > div > div.topbar_lft > span'
    # 종가/ 전일대비
    selector_close_price = '#svdMainGrid1 > table > tbody > tr.rwf > td:nth-child(2)'
    # 52주 최고가/최저가
    selector_one_year = '#svdMainGrid1 > table > tbody > tr:nth-child(2) > td:nth-child(2)'
    # 시가총액(보통주, 억원)
    selector_market_cap = '#svdMainGrid1 > table > tbody > tr:nth-child(5) > td:nth-child(2)'
    # 거래량
    selector_volume = '#svdMainGrid1 > table > tbody > tr.rwf > td.cle.r'
    # 거래대금(억원)
    selector_tr_amt = '#svdMainGrid1 > table > tbody > tr:nth-child(2) > td.cle.r'
    # 외국인 보유비중(%)
    selector_foreigner = '#svdMainGrid1 > table > tbody > tr:nth-child(3) > td.cle.r'

    v_ref_date = soup.select_one(selector_ref_date).get_text()
    v_close_price = soup.select_one(selector_close_price).get_text()
    v_one_year = soup.select_one(selector_one_year).get_text()
    v_market_cap = soup.select_one(selector_market_cap).get_text()
    v_volume = soup.select_one(selector_volume).get_text()
    v_tr_amt = soup.select_one(selector_tr_amt).get_text()
    v_foreigner = soup.select_one(selector_foreigner).get_text()

end = time.time()
dur1 = end - start    

### 2. 주식 가격 가져오기 (최근 4개월, 출처: KRX)

In [2]:
start = time.time()

year, month, day = v_ref_date.replace('[', '').replace(']', '').split('/')

ref_date = year + month + day                           # 기준일
dt_ref_date = datetime.strptime(ref_date, "%Y%m%d")
dt_past_date = dt_ref_date + relativedelta(months=-4)    # 4개월 전

date_from = dt_past_date.strftime('%Y%m%d')     # 주식 가격 가져올 시작 일자
date_to = ref_date                              # 주식 가격 가져올 마지막 일자

# pip install pykrx
from pykrx import stock

# 해당 종목의 지정한 기간의 주식 거래 현황
df_period_price = stock.get_market_ohlcv(date_from, date_to, ticker)
df_period_price = df_period_price.reset_index()
df_period_price = df_period_price.astype({'날짜' : 'string'})
max_row = df_period_price.shape[0]
df_recent_price = df_period_price.tail(5).sort_values('날짜', ascending=False).reset_index(drop=True)

end = time.time()
dur2 = end - start

### 3. Excel 문서 만들기

In [14]:
# import openpyxl
# wb = openpyxl.Workbook()
# if wb.epoch == openpyxl.utils.datetime.CALENDAR_WINDOWS_1900:
#     print("This workbook is using the 1900 date system.")

This workbook is using the 1900 date system.


In [17]:
start = time.time()

# pip install openpyxl
from openpyxl import Workbook
from openpyxl.chart import Reference, LineChart, BarChart
from openpyxl.styles import Font, Border, Side, PatternFill, Alignment, NamedStyle
from openpyxl.chart.axis import DateAxis

wb = Workbook()
# wb.iso_dates = True
# wb.epoch = openpyxl.utils.datetime.CALENDAR_MAC_1904
ws = wb.active  # 현재 활성화된 sheet 가져옴
ws.title = "DailyReport"

# 열 너비 지정
ws.column_dimensions["A"].width = 22     
ws.column_dimensions["B"].width = 8     
ws.column_dimensions["C"].width = 8    
ws.column_dimensions["D"].width = 18    
ws.column_dimensions["E"].width = 16    

# ws["A1"] = 1
ROW_TITLE = 1
ROW_NO_1 = 3
ROW_NO_2 = 9
ROW_NO_3 = 18

# 행 높이
ws.row_dimensions[ROW_TITLE].height = 30        # 제목 행 높이 지정
ws.row_dimensions[ROW_NO_1].height = 25        # 소제목 행 높이 지정
ws.row_dimensions[ROW_NO_1+1].height = 5       
ws.row_dimensions[ROW_NO_2].height = 25        # 소제목 행 높이 지정
ws.row_dimensions[ROW_NO_2+1].height = 5       
ws.row_dimensions[ROW_NO_3].height = 25        # 소제목 행 높이 지정
ws.row_dimensions[ROW_NO_3+1].height = 5       

title = ws.cell(row=ROW_TITLE, column=1)
sub_1 = ws.cell(row=ROW_NO_1, column=1)
sub_2 = ws.cell(row=ROW_NO_2, column=1)
sub_3 = ws.cell(row=ROW_NO_3, column=1)
ref_1 = ws.cell(row=ROW_NO_1, column=5)     # 출처 fnGuide

# Font 스타일 적용
title.font = Font(color="0000FF", size=17, bold=True)
sub_1.font = Font(size=13, italic=True, bold=True)
sub_2.font = Font(size=13, italic=True, bold=True)
sub_3.font = Font(size=13, italic=True, bold=True)
ref_1.font = Font(color="0000FF", size=9, underline="single")

# 정렬
title.alignment = Alignment(horizontal="left", vertical="center")
sub_1.alignment = Alignment(horizontal="left", vertical="center")
sub_2.alignment = Alignment(horizontal="left", vertical="center")
sub_3.alignment = Alignment(horizontal="left", vertical="center")
ref_1.alignment = Alignment(horizontal="right", vertical="bottom")

# 테두리 적용
thin_border = Border(left=Side(style="thin"),
                     right=Side(style="thin"),
                     top=Side(style="thin"),
                     bottom=Side(style="thin"))

for col in range(1, 6):
    ws.cell(row=ROW_NO_1+2, column=col).border = thin_border
    ws.cell(row=ROW_NO_1+3, column=col).border = thin_border
    ws.cell(row=ROW_NO_1+4, column=col).border = thin_border
    ws.cell(row=ROW_NO_2+2, column=col).border = thin_border
    ws.cell(row=ROW_NO_2+3, column=col).border = thin_border
    ws.cell(row=ROW_NO_2+4, column=col).border = thin_border
    ws.cell(row=ROW_NO_2+5, column=col).border = thin_border
    ws.cell(row=ROW_NO_2+6, column=col).border = thin_border
    ws.cell(row=ROW_NO_2+7, column=col).border = thin_border
    
# 셀 병합
ws.merge_cells("B5:C5")
ws.merge_cells("B6:C6")
ws.merge_cells("B7:C7")

ws.cell(row=ROW_TITLE, column=1, value="대성에너지 일일 주가 동향 "+v_ref_date)

ws.cell(row=ROW_NO_1, column=1, value="1. 일일 현황")
ws.cell(row=ROW_NO_1, column=5, value="(출처: fnGuide)")

ws.cell(row=ROW_NO_1+2, column=1, value="종가/ 전일대비")
ws.cell(row=ROW_NO_1+2, column=2).value = v_close_price
ws.cell(row=ROW_NO_1+2, column=2).alignment = Alignment(horizontal="right", vertical="center")
ws.cell(row=ROW_NO_1+3, column=1, value="52주 최고/최저")
ws.cell(row=ROW_NO_1+3, column=2).value = v_one_year
ws.cell(row=ROW_NO_1+3, column=2).alignment = Alignment(horizontal="right", vertical="center")
ws.cell(row=ROW_NO_1+4, column=1, value="시가총액(보통주, 억원)")
ws.cell(row=ROW_NO_1+4, column=2).value = v_market_cap
ws.cell(row=ROW_NO_1+4, column=2).alignment = Alignment(horizontal="right", vertical="center")

ws.cell(row=ROW_NO_1+2, column=4, value="거래량")
ws.cell(row=ROW_NO_1+2, column=5).value = v_volume
ws.cell(row=ROW_NO_1+2, column=5).alignment = Alignment(horizontal="right", vertical="center")
ws.cell(row=ROW_NO_1+3, column=4, value="거래대금(억원)")
ws.cell(row=ROW_NO_1+3, column=5).value=v_tr_amt
ws.cell(row=ROW_NO_1+3, column=5).alignment = Alignment(horizontal="right", vertical="center")
ws.cell(row=ROW_NO_1+4, column=4, value="외국인 보유비중(%)")
ws.cell(row=ROW_NO_1+4, column=5).value = v_foreigner
ws.cell(row=ROW_NO_1+4, column=5).alignment = Alignment(horizontal="right", vertical="center")

for row in range(ROW_NO_1+2, ROW_NO_1+5):
    ws.cell(row=row, column=1).fill = PatternFill(fgColor="99ccff", fill_type="solid")
    ws.cell(row=row, column=4).fill = PatternFill(fgColor="99ccff", fill_type="solid")


ws.cell(row=ROW_NO_2, column=1, value="2. 최근 일주일 거래 현황")

ws.cell(row=ROW_NO_2+2, column=1, value='날짜')
ws.cell(row=ROW_NO_2+2, column=2, value='종가')
ws.cell(row=ROW_NO_2+2, column=3, value='등락률')
ws.cell(row=ROW_NO_2+2, column=4, value='거래량')
ws.cell(row=ROW_NO_2+2, column=5, value='거래대금')
for col in range(1, 6):
    ws.cell(row=ROW_NO_2+2, column=col).alignment = Alignment(horizontal="center", vertical="center")
    ws.cell(row=ROW_NO_2+2, column=col).fill = PatternFill(fgColor="99ccff", fill_type="solid")
    
for idx, data in df_recent_price.iterrows():
    ws.cell(row=idx+ROW_NO_2+3, column=1).value = data['날짜']
    ws.cell(row=idx+ROW_NO_2+3, column=1).alignment = Alignment(horizontal="center", vertical="center")
    ws.cell(row=idx+ROW_NO_2+3, column=2).value = data['종가']
    ws.cell(row=idx+ROW_NO_2+3, column=2).alignment = Alignment(horizontal="right", vertical="center")
    ws.cell(row=idx+ROW_NO_2+3, column=2).number_format = '#,##0'
    ws.cell(row=idx+ROW_NO_2+3, column=3).value = data['등락률']
    ws.cell(row=idx+ROW_NO_2+3, column=3).alignment = Alignment(horizontal="right", vertical="center")
    ws.cell(row=idx+ROW_NO_2+3, column=4).value = data['거래량']
    ws.cell(row=idx+ROW_NO_2+3, column=4).alignment = Alignment(horizontal="right", vertical="center")
    ws.cell(row=idx+ROW_NO_2+3, column=4).number_format = '#,##0'
    ws.cell(row=idx+ROW_NO_2+3, column=5).value = data['거래대금']
    ws.cell(row=idx+ROW_NO_2+3, column=5).alignment = Alignment(horizontal="right", vertical="center")
    ws.cell(row=idx+ROW_NO_2+3, column=5).number_format = '#,##0'

ws.cell(row=ROW_NO_3, column=1, value="3. 주가 및 거래량 추이 (최근 4개월)")

""" BEGIN: 데이터 Sheet """

ws_data = wb.create_sheet("DataSheet", 2)        # 2번째 index에 sheet 생성

# 열 너비 지정
ws_data.column_dimensions["A"].width = 15     
ws_data.column_dimensions["B"].width = 15    
ws_data.column_dimensions["C"].width = 15   
ws_data.column_dimensions["D"].width = 15    
ws_data.column_dimensions["E"].width = 25  

# 1줄씩 데이터 넣기
ws_data.append(["날짜", "종가", "등락률", "거래량", "거래대금"])
for index, data in df_period_price.iterrows():
    ws_data.append([data['날짜'], data['종가'], data['등락률'], data['거래량'], data['거래대금']])

# date_style = Style(number_format="YYYY-MM-DD HH:MM:MM")
date_style = NamedStyle(name='date_style', number_format='YYYY-MM-DD')
col_A = ws_data["A"]
for cell in col_A:
    cell.style = date_style
    
""" END: 데이터 Sheet """

# 종가
# value_1 = Reference(ws_data, min_row=1, max_row=max_row+1, min_col=2, max_col=2)
value_1 = Reference(ws_data, min_row=1, max_row=10, min_col=2, max_col=2)
# 거래량
# value_2 = Reference(ws_data, min_row=1, max_row=max_row+1, min_col=4, max_col=4)
value_2 = Reference(ws_data, min_row=1, max_row=10, min_col=4, max_col=4)
# 날짜
# category = Reference(ws_data, min_row=1, max_row=max_row+1, min_col=1)
category = Reference(ws_data, min_row=1, max_row=10, min_col=1)

line_chart = LineChart()
line_chart.set_categories(category)
line_chart.add_data(value_1, titles_from_data=True)
# line_chart.title = None
line_chart.legend = None
line_chart.style = 10
line_chart.height = 6
line_chart.x_axis.delete = True
# line_chart.x_axis.number_format = 'd-mmm' ## 일 - 월 세글자만
# line_chart.x_axis.title = "날짜"
# line_chart.y_axis.title = "종가"
ws.add_chart(line_chart, "A20")   # 차트 넣을 위치

bar_chart = BarChart()
bar_chart.set_categories(category)
bar_chart.add_data(value_2, titles_from_data=True)
# bar_chart.title = None
bar_chart.legend = None
bar_chart.height = 6
# bar_chart.auto_axis = False
# bar_chart.y_axis.unit = 2500000
# bar_chart.x_axis.delete = True
# bar_chart.y_axis.crossAx = 1000
# bar_chart.x_axis = DateAxis(crossAx=200)
bar_chart.x_axis.number_format ='yyyy/mm/dd'
# bar_chart.x_axis.majorTimeUnit = "days"
# bar_chart.x_axis.number_format = 'd-mmm' ## 일 - 월 세글자만
# bar_chart.x_axis.number_format ='yyyy-mm-dd'
# bar_chart.x_axis.title = "날짜"
# bar_chart.y_axis.title = "거래량"
ws.add_chart(bar_chart, "A31")   # 차트 넣을 위치

# 파일명은 full path로 지정
excel_file = "D:\PythonProject\data-gatherer\sample\data\dse_daily_price.xlsx"
wb.save(excel_file)
wb.close()

end = time.time()
dur3 = end - start

### 4. Excel 문서를 PDF로 변환하기

In [70]:
start = time.time()

import win32com.client

excel = win32com.client.DispatchEx("Excel.Application")
excel.Visible = False
    
wb = excel.Workbooks.Open(excel_file)
ws = wb.ActiveSheet

# Type=0 "PDF", Type=1 "XPS"
pdf_file = "D:\PythonProject\data-gatherer\sample\data\dse_daily_price.pdf"
ws.ExportAsFixedFormat(Type=0, Filename=pdf_file)    

wb.application.displayalerts = False  # This stops the popup asking for a save
wb.Close()

excel.Quit()

end = time.time()
dur4 = end - start

### 5. 이메일 발송

In [75]:
start = time.time()

import csv

# Email 수신자 목록
receiver_file = "D:\PythonProject\data-gatherer\sample\data\sample_email_receiver.csv"
f = open(receiver_file, 'r', encoding='utf-8')
lines = csv.reader(f)
receiver_list = []
for line in lines:
    # print(line)
    receiver_list.append(line[0])
f.close()

""" Send email with attachment """
"""
<< Prerequisites >>
구글 이메일 계정 사용
1. 보안 - 2단계 인증 사용 설정
2. 앱 비밀번호 생성: 앱-메일, 기기-Windows 컴퓨터 선택 => 앱 비밀번호 생성

"""
EMAIL_ADDRESS = "kyungho.lim@gmail.com"
EMAIL_PASSWORD = "zqwsuqdrtmnzvccx"   # Windows 컴퓨터용 앱 비밀번호

import smtplib
# from account import *
from email.message import EmailMessage
import os

msg = EmailMessage()

msg["Subject"] = "대성에너지 일일 주가 동향"+v_ref_date

msg["From"] = EMAIL_ADDRESS

# 수신자가 여러명일 경우 콤마(,)로 구분하여 문자열 생성, msg["To"]에 할당
msg["To"] = ', '.join(receiver_list) 

# 본문
msg.set_content("대성에너지 일일 주가 동향"+v_ref_date+"입니다. 첨부파일을 다운로드 하십시오.")

"""
MIME type 전체 목록 참조하여 보내는 파일형식에 맞는 maintype과 subtype 지정
=> https://developer.mozilla.org/ko/docs/Web/HTTP/Basics_of_HTTP/MIME_types/Common_types
"""
with open("D:\PythonProject\data-gatherer\sample\data\dse_daily_price.pdf", "rb") as f:
    # msg.add_attachment(f.read(), maintype="application", subtype="pdf", filename=f.name)
    file_name = os.path.basename(f.name)
    msg.add_attachment(f.read(), maintype="application", subtype="pdf", filename=file_name)

# 메일 발송
with smtplib.SMTP("smtp.gmail.com", 587) as smtp:   # port no. 587
    smtp.ehlo()         # 연결이 잘 수립되는지 확인
    smtp.starttls()     # 모든 내용이 암호화되어 전송
    smtp.login(EMAIL_ADDRESS, EMAIL_PASSWORD)   # 로그인
    
    smtp.send_message(msg)
    
end = time.time()
dur5 = end - start

### [실행시간]

In [77]:
print("총 소요시간")
print(f"1. 주식 정보 가져오기(fnGuide)\t{dur1:.5f} sec")
print(f"2. 주식 가격 가져오기(최근 4개월)\t{dur2:.5f} sec")
print(f"3. Excel 문서 만들기\t\t{dur3:.5f} sec")
print(f"4. Excel 문서를 PDF로 변환하기\t{dur4:.5f} sec")
print(f"5. 이메일 발송\t\t\t{dur5:.5f} sec")
print(f"** 총 소요시간\t\t\t{dur1+dur2+dur3+dur4+dur5:.5f} sec")

총 소요시간
1. 주식 정보 가져오기(fnGuide)	0.14071 sec
2. 주식 가격 가져오기(최근 4개월)	0.25741 sec
3. Excel 문서 만들기		0.03179 sec
4. Excel 문서를 PDF로 변환하기	3.54781 sec
5. 이메일 발송			4.65781 sec
** 총 소요시간			8.63554 sec
